In [4]:
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns


In [5]:
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12,10)

In [ ]:
page_133 = requests.get('http://193.246.68.117:8080/gp/GP?type=EXPORT&indikatoren=133&raumtyp=1&text=yes')

In [11]:
def api_get_dataframe(url_: str) -> (pd.DataFrame, pd.DataFrame):
    json_ = requests.get(url_).json()
    return (pd.io.json.json_normalize(json_['RAUM']),
            pd.io.json.json_normalize(json_['RAUM']))

In [7]:
raum, indi = api_get_dataframe('http://193.246.68.117:8080/gp/GP?type=EXPORT&indikatoren=133&raumtyp=1&text=yes')

In [14]:
def gemeinde_dataframe(raum_df_: pd.DataFrame) -> pd.DataFrame:
    raum_dates = raum_df_.drop(['BFS', 'DATEN_VORHANDEN', 'ALLE_JAHRE', 'RAUMEINHEIT_ID', 'RAUMEINHEIT_NAME'], axis=1).astype(float)
    raum_dates.columns = raum_df_.ALLE_JAHRE[0]
    raum_dates.index = raum_df_.RAUMEINHEIT_NAME
    return raum_dates

In [15]:
def gemeinde_by_date(raum_df_: pd.DataFrame, gemeinde: str) -> pd.DataFrame:
    df = raum_df_[raum_df_.RAUMEINHEIT_NAME == gemeinde]
    ix = df.index[0]
    years = df.ALLE_JAHRE.values[0]
    df = df.drop(['BFS', 'DATEN_VORHANDEN', 'ALLE_JAHRE', 'RAUMEINHEIT_ID','RAUMEINHEIT_NAME'], axis=1).astype(float)
    df.columns = years
    return pd.DataFrame(df.stack()[ix], columns=[gemeinde])